## Connect all the seismic station with GDAL (Only python < 3.8)

In [1]:
from osgeo import gdal, ogr, osr
import csv

In [2]:
root = 'D:\\MEMOIRE\\SIG\\Stations\\Julien QGIS\\'
rootcsv = 'D:\\MEMOIRE\\MSNoise\\ablation\\DTT\\02\\030_DAYS\\ZZ\\'

In [3]:
dtt_day = '2021-05-20'

In [4]:
PAIRDTT = []
DTT = []
with open(rootcsv+dtt_day+'.txt') as file :
    readcsv = csv.reader(file, delimiter=',')
    line = 0
    for row in readcsv :
        if line == 0 :
            p = row.index('Pairs')
            m = row.index('M0')
        else :
            PAIRDTT.append(row[p])
            DTT.append(float(row[m])*100)            
        line += 1
print(PAIRDTT,DTT)

['VI_BJK_VI_GRF', 'VI_DJK_VI_GRF', 'VI_GRF_VI_HUS', 'VI_GRF_VI_JOK', 'VI_GRF_VI_KSK', 'VI_GRF_VI_URH', 'VI_GRF_VI_VOT', 'ALL', 'VI_BJK_VI_DJK', 'VI_BJK_VI_HUS', 'VI_BJK_VI_JOK', 'VI_BJK_VI_KSK', 'VI_BJK_VI_URH', 'VI_BJK_VI_VOT', 'VI_DJK_VI_HUS', 'VI_DJK_VI_JOK', 'VI_DJK_VI_KSK', 'VI_DJK_VI_URH', 'VI_DJK_VI_VOT', 'VI_HUS_VI_JOK', 'VI_HUS_VI_KSK', 'VI_HUS_VI_URH', 'VI_HUS_VI_VOT', 'VI_JOK_VI_KSK', 'VI_JOK_VI_URH', 'VI_JOK_VI_VOT', 'VI_KSK_VI_URH', 'VI_KSK_VI_VOT', 'VI_URH_VI_VOT'] [-0.0378541446606, -0.10270940444900001, -0.13502033528000001, -0.12954244699099998, -0.0332787480377, -0.108431184681, -0.07122066216259999, -0.0681862635504, -0.18818216670100002, -0.127729123656, -0.061558883268, -0.095814921422, -0.0409595136154, -0.0221711071452, -0.174361017857, 0.0608817956526, -0.0605341287273, -0.114267993404, -0.178184610381, -0.15881788059500002, -0.106279922629, -0.0648953530049, -0.0855686394319, -0.07591377335260001, 0.0744941857937, 0.06430238975989999, -0.19881696307399999, -0.1

In [5]:
NET = 'VI'
STA = ['BJK','DJK','HUS','VOT','JOK','KSK','URH','GRF']
STA.sort()

In [6]:
metadata = []
with open(root+'stations.txt') as file :
    readcsv = csv.reader(file, delimiter='|')
    line = 0
    for row in readcsv :
        if line == 0 :
            stt_idx = row.index('Station')
            lat_idx = row.index('Latitude')
            long_idx = row.index('Longitude')
        elif row[stt_idx].upper() in STA :
            metadata.append([row[stt_idx].upper(),row[lat_idx],row[long_idx]])
            
        line += 1

In [7]:
print(metadata)

[['BJK', '64.46911', '-16.75205'], ['DJK', '64.50386', '-17.23485'], ['GRF', '64.40656', '-17.26713'], ['HUS', '64.2955', '-17.6565'], ['JOK', '64.31529', '-18.22546'], ['KSK', '64.1613', '-16.4737'], ['URH', '64.820373', '-17.147123'], ['VOT', '64.27081', '-17.18254']]


In [8]:
line = ogr.Geometry(ogr.wkbLineString)
line.AddPoint(1116651.439379124, 637392.6969887456)
line.AddPoint(1188804.0108498496, 652655.7409537067)
line.AddPoint(1226730.3625203592, 634155.0816022386)
line.AddPoint(1281307.30760719, 636467.6640211721)
print (line.ExportToWkt())

LINESTRING (1116651.43937912 637392.696988746 0,1188804.01084985 652655.740953707 0,1226730.36252036 634155.081602239 0,1281307.30760719 636467.664021172 0)


In [10]:
import osgeo.ogr as ogr
import osgeo.osr as osr

# set up the shapefile driver
driver = ogr.GetDriverByName("ESRI Shapefile")

# create the data source
data_source = driver.CreateDataSource("D:/HUB_dtt.shp")

# create the spatial reference, WGS84
srs = osr.SpatialReference()
srs.ImportFromEPSG(4326)

# create the layer
layer = data_source.CreateLayer("HUB", srs, ogr.wkbLineString)

# Add the fields we're interested in
field_name = ogr.FieldDefn("Name", ogr.OFTString)
field_name.SetWidth(24)
layer.CreateField(field_name)
field_dtt = ogr.FieldDefn("dtt", ogr.OFTReal)
layer.CreateField(field_dtt)


# Process the text file and add the attributes and features to the shapefile
for i in range(len(STA)-1) :
    for j in range(i+1,len(STA)):
        pair = "%s_%s_%s_%s" % (NET,metadata[i][0],NET,metadata[j][0])
        if pair in PAIRDTT :   
          # create the feature
            feature = ogr.Feature(layer.GetLayerDefn())
          # Set the attributes using the values from the delimited text file
            feature.SetField("Name", pair)
            feature.SetField("dtt",DTT[PAIRDTT.index(pair)]) 

          # create the WKT for the feature using Python string formatting
            wkt = "LINESTRING(%f %f 0,%f %f 0)" %  (float(metadata[i][2]) , float(metadata[i][1]), float(metadata[j][2]) , float(metadata[j][1]))

          # Create the point from the Well Known Txt
            line = ogr.CreateGeometryFromWkt(wkt)

          # Set the feature geometry using the point
            feature.SetGeometry(line)
          # Create the feature in the layer (shapefile)
            layer.CreateFeature(feature)
          # Dereference the feature
            feature = None

# Save and close the data source
data_source = None